In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
from scipy import integrate 
import networkx as nx

## Frank-Wolfe Algorithm Implementation for the User Equilibrium
The following code implements the Frank-Wolfe Algorithm that is applied to the Sioux Falls South Dakota network to determine user-optimized links flows.

We first import data related to the link performance function parameters as well as the origin-destination matrix elements.
The next step defines the links performance function, the user equilibrium objective, and the step size functions

In [2]:
demand=pd.read_csv("demand.csv",header=None)   
demand=np.array(demand)

linksdata=pd.read_csv("links.csv",header=None)     
linksdata=np.array(linksdata)

In [3]:
def traveltime(t0,xa,ca):
    ta=t0*(1+0.15*(xa/ca)**4)
    return ta

def objectiveZ(alpha,xa,ca,t0,ya):
    Z=0
    for i in range(len(xa)):
        Z+=integrate.quad(lambda x: traveltime(t0[i],x,ca[i]),0,xa[i]+alpha*(ya[i]-xa[i]))[0]
    return Z

def stepsize(xa,ca,t0,ya):
    alpha=minimize_scalar(lambda alpha: objectiveZ(alpha,xa,ca,t0,ya),bounds=(0,1),method='Bounded')
    return alpha.x

The Frank-Wolfe algorithm implementation can be found below:

In [4]:
#Initialization steps
t0=linksdata[:,0]                                    
ca=linksdata[:,1]                                     
ta=t0
#Read an adjacency matrix from the appropriate csv file
adjacency=pd.read_csv("adjacency.csv",header=None)   
adjacency=np.array(adjacency)
#Use of Network X package to create the network topology
G=nx.DiGraph()                                      
G=nx.from_numpy_matrix(adjacency,parallel_edges=True,create_using=nx.DiGraph) 
links=np.array(G.edges())                            
xa=np.zeros(len(links))                              #link flow at step n
xa1=np.zeros(len(links))                             #link flow at step n+1
eps=1                                                #epsilon for convergence check

#Block for Frank-Wolfe algorithm
count=0
while eps >= 0.001:                                  #stop when epsilon is approximately 0.001
    count+=1                                         #number of iterations
    for i in range(len(links)):
        xa[i]=xa1[i]                                 #revisit link flow at the next iteration (xa(n) is set as xa(n+1))
    if(count == 1):
        ta=traveltime(t0,xa,ca)
        for k in range(len(ta)):
            adjacency[links[k][0],links[k][1]] = ta[k]                  #Djikstra with weightes as the travel time values
        G = nx.DiGraph()
        G = nx.from_numpy_matrix(adjacency,parallel_edges=True,create_using=nx.DiGraph)
        path = dict(nx.all_pairs_dijkstra_path(G,weight='weight'))      #Djikstra for all node pairs in the network 
        for i in range(len(adjacency)):
            for j in range(len(adjacency)):                             #in which shortest paths a specific link is utilized
                if(i!=j):                                               #capturing this for all links
                    tempPath = path[i][j]                               #recalculating the travel times with new link flows
                    for k in range(len(tempPath)-1):
                        temp2 = [tempPath[k], tempPath[k+1]]            
                        for count2 in range(len(links)):
                            if(temp2[0] == links[count2][0] and temp2[1] == links[count2][1]):
                                xa[count2] = xa[count2] + demand[i][j]     #updating link flow according to the O-D pair demand
    ta = traveltime(t0,xa,ca)
    for k in range(len(ta)):
        adjacency[links[k][0],links[k][1]] = ta[k]                          #in every iteartion recreating the adjacency matrix
    G=nx.DiGraph()                                                          #where links are weighted by linke performance in matrix
    G=nx.from_numpy_matrix(adjacency,parallel_edges=True,create_using=nx.DiGraph)  #then generate weighted network
    path = dict(nx.all_pairs_dijkstra_path(G,weight='weight'))              #again, all node pairs shortest paths are computed
    ya=np.zeros(len(links))
    for i in range(len(adjacency)):
        for j in range(len(adjacency)):                                     #which links appear in which O-D pairs' shortest path
            if(i!=j):                                                       #are recorded and ya is assigned with flow of that O-D
                tempPath = path[i][j]
                for k in range(len(tempPath)-1):
                    temp2 = [tempPath[k], tempPath[k+1]]
                    for count2 in range(len(links)):
                        if(temp2[0] == links[count2][0] and temp2[1] == links[count2][1]):
                            ya[count2] = ya[count2] + demand[i][j]        #after ya values are assigned, alpha is calculated
    alpha=stepsize(xa,ca,t0,ya)
    eps1=np.zeros(len(links))
    sum1=0
    for i in range(len(links)):
        xa1[i]=xa[i]+alpha*(ya[i]-xa[i])                                   #xa(n+1) is updated by optimum alpha, xa(n), and ya
        eps1[i]=(xa1[i]-xa[i])**2
        sum1+=xa[i]
    eps=(sum(eps1)**0.5)/sum1
    print(eps)                                                             #convergence after each iteration

0.016536932703591842
0.018683348611280407
0.005997686929020705
0.009302603191124815
0.003530658653254738
0.0012505213946284676
0.0006977684578352756


In [5]:
#Final results are printed below:
print('convergence is:', eps) 
print('number of iterations is:', count)
print('alpha is:', alpha)
print('final UE travel times are:', ta)  
print('final UE link flows are:', xa1)

convergence is: 0.0006977684578352756
number of iterations is: 7
alpha is: 0.03127209128100375
final UE travel times are: [3.76966814 3.32032031 3.60884402 3.12671829 2.40135763 2.40087024
 2.40135067 2.40299031 1.21320539 3.74304119 1.20184624 2.45262267
 3.0723888  3.78209718 2.60215523 1.21174549 1.80268164 1.20002251
 1.45771954 1.80268164 2.00073508 3.00840394 3.00626318 2.00049929
 1.80283613 1.80283613 3.00163156 3.60479811 3.00032997 4.20126308
 3.70272802 3.00162654 3.62821845 3.04273807 2.40135067 3.62821845
 1.80046811 1.80046811 2.78169168 3.04187694 3.01697217 2.54812282
 3.60480686 3.01444515 2.43611128 2.41626072 3.00840394 3.00032997
 1.20291059 1.80000092 4.20126308 1.20291059 1.22420917 1.20002251
 1.80000092 2.56176754 2.83449486 1.22420917 2.4302747  2.56176755
 2.52011735 3.65720109 3.00433496 3.64299792 1.29186566 2.30909065
 2.42091342 3.00244292 1.27153887 2.41314511 2.56411627 2.40947957
 1.21375048 2.64441887 2.3341245  1.2123815 ]
final UE link flows are: [ 4

In [6]:
print(sum(xa1*ta)) #computing the total travel time in the transportation network under UE conditions

1270801.7962069164
